In [9]:
from jetbot import Camera
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
from jetbot import bgr8_to_jpeg
import torch
import torchvision
import ipywidgets
import torchvision.models as models
from utils import preprocess
import torch.nn.functional as F
import time
from jetbot import Robot

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg',width=224, height=224)
circle_slider = widgets.FloatSlider(description='circle',min=0.0, max=1.0, orientation='vertical')
square_slider = widgets.FloatSlider(description='square',min=0.0, max=1.0, orientation='vertical')


model = models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

model.load_state_dict(torch.load('my_model.pth'))
device = torch.device('cuda')
model = model.to(device)


robot = Robot()

def update(change) :
    global blocked_slider, robot
    x = change['new']
    x = preprocess(x)
    y = model(x)
    
    y = F.softmax(y, dim=1)
    print(y)
    prob_circle = float(y.flatten()[1])
    prob_square = float(y.flatten()[0])
    
    circle_slider.value = prob_circle
    square_slider.value = prob_square
    
    if prob_circle > 0.6 :
        time.sleep(5)
        robot.forward(1)
        time.sleep(3)
        robot.stop()
        robot.up(1)
        time.sleep(3)
        robot.stop()
        robot.right(0.75)
        time.sleep(3)
        robot.stop()
        robot.backward(1)
        time.sleep(3)
        robot.stop()
        robot.down(1)
        time.sleep(3)
        robot.stop()
    if prob_square > 0.6 :
        robot.set_motors(1, 1)
        robot.right(0.5)
        time.sleep(14)
    robot.stop()
    

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(widgets.HBox([image, circle_slider, square_slider]))
update({'new': camera.value})
camera.observe(update, names='value')


tensor([[9.9939e-01, 6.0529e-04]], device='cuda:0', grad_fn=<SoftmaxBackward>)


In [4]:
robot.stop()